# Web Scraping for Data (Interactivity)

## Using an interactive, automated browser

In [1]:
# Using Splinter, an automated solution to do interactive web scraping
from splinter import Browser

# Using Chrome as our web browser
from webdriver_manager.chrome import ChromeDriverManager

# Initialize Chrome (must already have Chrome installed)
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

# Point Chrome to the website we want to visit
url = "https://books.toscrape.com/index.html"
browser.visit(url)

# Grab the HTML from the web page
html = browser.html



====== WebDriver manager ======
Could not get version for google-chrome.Is google-chrome installed?
Get LATEST chromedriver version for None google-chrome
Driver [C:\Users\Charl\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [2]:
# Import BeautifulSoup
from bs4 import BeautifulSoup

# Parse the HTML from our automated Chrome browser using the lxml engine
soup = BeautifulSoup(html, "lxml")

# Grab the sidebar from the page and target the <ul> tag that contains all of the category links
side_bar = soup.select_one("ul.nav.nav-list").ul

# Grab all links from the list of categories
categories = side_bar.select("a")

print(categories)

[<a href="catalogue/category/books/travel_2/index.html">
                            
                                Travel
                            
                        </a>, <a href="catalogue/category/books/mystery_3/index.html">
                            
                                Mystery
                            
                        </a>, <a href="catalogue/category/books/historical-fiction_4/index.html">
                            
                                Historical Fiction
                            
                        </a>, <a href="catalogue/category/books/sequential-art_5/index.html">
                            
                                Sequential Art
                            
                        </a>, <a href="catalogue/category/books/classics_6/index.html">
                            
                                Classics
                            
                        </a>, <a href="catalogue/category/books/phil

In [3]:
# Create empty dictionary to store our data (used later)
book_info = {
    "title": [],
    "url": [],
    "num_stars": [],
    "price_gbp": [],
    "availability": []
}

# Dictionary to translate star ratings to numeric ratings
ratings = {
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5
}

# For each category link in our list of links...
for category in categories:
    # Start a new counter to keep track of each book in the category
    count = 1
    
    # Get the text of the category from the link
    category_name = category.text.strip()

    # Click on the appropriate link based on the category link
    browser.links.find_by_partial_href(category["href"].split("/")[-2]).click()

    # Get the new HTML of the category from the browser
    html = browser.html

    # Parse the new HTML with beautifulsoup
    soup = BeautifulSoup(html, "lxml")

    # Get the book containers in the category
    books = soup.find_all("article", class_="product_pod")
    
    # Grab total number of books in the category
    total_in_category = soup.select_one("form.form-horizontal")\
        .text.strip().split()[0]

    # Try to see if there are additional pages...
    try:
        num_pages = int(soup.find("li", class_="current").text.split()[-1])

    # If we can't find any, then we know we only have this one page
    except:
        # For each book container on the page...
        for book in books:
            # Because the index.html for each book's web page is stored
            # several layers down in the website's directory structure,
            # we need to grab the important parts of the URL (the directory
            # containing the index.html and the html file itself) in order to
            # reconstruct the absolute URL path later on.
            path = (
                f"{book.a['href'].split('/')[-2]}/"
                f"{book.a['href'].split('/')[-1]}"
            )

            # Category name
            print(f"Category: {category_name}")

            # Print book count out of total books in the current category
            print(f"Book {count} of {total_in_category}: {book.h3.a['title']}")
            book_info["title"].append(book.h3.a["title"])

            # Reconstruct the URL to the book's web page
            book_url = f"books.toscrape.com/catalogue/{path}"
            print(f"Link: {book_url}")
            book_info["url"].append(book_url)
            
            # Print rating from class name
            num_stars = book.p["class"][1].lower()
            print(f"Rating (Number of Stars): {ratings[num_stars]}")
            book_info["num_stars"].append(ratings[num_stars])
            
            # Print price of book (account for £ symbol)
            price = book.select_one("p.price_color").text.strip()[1:]
            print(f"Price (Pound Sterling): {price}")
            book_info["price_gbp"].append(price)
            
            # Print availability of book
            status = book.select_one("p.availability").text.strip()
            print(f"Availability: {status}")
            book_info["availability"].append(status)
            
            # Quick and dirty line separator
            print("-" * 80)

            # Increment the book count in this category
            count += 1
        
        # We are finished with the books in this category once loop ends
        print("No more books in this category.")
        print("-" * 80)

    # However, if there are more pages...
    else:
        # If we're in this section, num_pages must have a value, 
        # so we iterate through these pages in the current category...
        for page in range(num_pages):
            # Get each book container and print their information like before
            for book in books:
                # Grab important parts of each book's URL to reconstruct into
                # an absolute path (see explanation in the "except" block)
                path = (
                    f"{book.a['href'].split('/')[-2]}/"
                    f"{book.a['href'].split('/')[-1]}"
                )

                # Category name
                print(f"Category: {category_name}")

                # Page count, account for range starting at 0
                print(f"Page: {page+1}")
                
                # Print book count out of total books in the current category
                print(
                    f"Book {count} of {total_in_category}: "
                    f"{book.h3.a['title']}"
                )
                book_info["title"].append(book.h3.a["title"])

                # Reconstruct the URL to the book's web page
                book_url = f"books.toscrape.com/catalogue/{path}"
                print(f"Link: {book_url}")
                book_info["url"].append(book_url)

                # Print rating from class name
                num_stars = book.p["class"][1].lower()
                print(f"Rating (Number of Stars): {ratings[num_stars]}")
                book_info["num_stars"].append(ratings[num_stars])

                # Print price of book (account for £ symbol)
                price = book.select_one("p.price_color").text.strip()[1:]
                print(f"Price (Pound Sterling): {price}")
                book_info["price_gbp"].append(price)

                # Print availability of book
                status = book.select_one("p.availability").text.strip()
                print(f"Availability: {status}")
                book_info["availability"].append(status)
                
                # Quick and dirty line separator
                print("-" * 80)

                # Increment the book count in this category
                count += 1

            # Prepare the next page in the category to use in the next loop
            try:
                # Try to click on the "next" button if we can...
                browser.links.find_by_partial_text("next").click()
                
            # If we can't get a new page, then we know we're on the last page
            # of the category
            except:
                print("No more books in this category.")
                print("-" * 80)

            # Otherwise if we can, load the new HTML and books
            else:
                html = browser.html
                soup = BeautifulSoup(html, "lxml")
                books = soup.find_all("article", class_="product_pod")

# Once we exit the loop, that means all categories have been scraped
print("Scraping completed.")

# Close the browser
browser.quit()

Category: Travel
Book 1 of 11: It's Only the Himalayas
Link: books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 45.17
Availability: In stock
--------------------------------------------------------------------------------
Category: Travel
Book 2 of 11: Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond
Link: books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 49.43
Availability: In stock
--------------------------------------------------------------------------------
Category: Travel
Book 3 of 11: See America: A Celebration of Our National Parks & Treasured Sites
Link: books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 48.87
Availability: In stock
----------------------------------------------

Category: Mystery
Page: 2
Book 21 of 32: The Mysterious Affair at Styles (Hercule Poirot #1)
Link: books.toscrape.com/catalogue/the-mysterious-affair-at-styles-hercule-poirot-1_452/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 24.80
Availability: In stock
---------------------------------------------------------------------------
Category: Mystery
Page: 2
Book 22 of 32: In the Woods (Dublin Murder Squad #1)
Link: books.toscrape.com/catalogue/in-the-woods-dublin-murder-squad-1_433/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 38.38
Availability: In stock
---------------------------------------------------------------------------
Category: Mystery
Page: 2
Book 23 of 32: The Silkworm (Cormoran Strike #2)
Link: books.toscrape.com/catalogue/the-silkworm-cormoran-strike-2_280/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 23.05
Availability: In stock
---------------------------------------------------------------------------
Category: Mystery

No more books in this category.
---------------------------------------------------------------------------
Category: Sequential Art
Page: 1
Book 1 of 75: Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
Link: books.toscrape.com/catalogue/scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 52.29
Availability: In stock
---------------------------------------------------------------------------
Category: Sequential Art
Page: 1
Book 2 of 75: Tsubasa: WoRLD CHRoNiCLE 2 (Tsubasa WoRLD CHRoNiCLE #2)
Link: books.toscrape.com/catalogue/tsubasa-world-chronicle-2-tsubasa-world-chronicle-2_949/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 16.28
Availability: In stock
---------------------------------------------------------------------------
Category: Sequential Art
Page: 1
Book 3 of 75: This One Summer
Link: books.toscrape.com/catalogue/this-one-summer_947/index.html
Rating (Number of Stars): 4
Price (Poun

Category: Sequential Art
Page: 3
Book 41 of 75: Fruits Basket, Vol. 7 (Fruits Basket #7)
Link: books.toscrape.com/catalogue/fruits-basket-vol-7-fruits-basket-7_468/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 19.57
Availability: In stock
---------------------------------------------------------------------------
Category: Sequential Art
Page: 3
Book 42 of 75: Fruits Basket, Vol. 6 (Fruits Basket #6)
Link: books.toscrape.com/catalogue/fruits-basket-vol-6-fruits-basket-6_427/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 20.96
Availability: In stock
---------------------------------------------------------------------------
Category: Sequential Art
Page: 3
Book 43 of 75: Death Note, Vol. 6: Give-and-Take (Death Note #6)
Link: books.toscrape.com/catalogue/death-note-vol-6-give-and-take-death-note-6_425/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 36.39
Availability: In stock
---------------------------------------------------------------

No more books in this category.
---------------------------------------------------------------------------
Category: Classics
Book 1 of 19: The Secret Garden
Link: books.toscrape.com/catalogue/the-secret-garden_413/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 15.08
Availability: In stock
--------------------------------------------------------------------------------
Category: Classics
Book 2 of 19: The Metamorphosis
Link: books.toscrape.com/catalogue/the-metamorphosis_409/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 28.58
Availability: In stock
--------------------------------------------------------------------------------
Category: Classics
Book 3 of 19: The Pilgrim's Progress
Link: books.toscrape.com/catalogue/the-pilgrims-progress_353/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 50.26
Availability: In stock
--------------------------------------------------------------------------------
Category: Classics
Book 4 of 19: The Hou

...

Category: Parenting
Book 1 of 1: Catastrophic Happiness: Finding Joy in Childhood's Messy Years
Link: books.toscrape.com/catalogue/catastrophic-happiness-finding-joy-in-childhoods-messy-years_138/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 37.35
Availability: In stock
--------------------------------------------------------------------------------
No more books in this category.
---------------------------------------------------------------------------
Category: Adult Fiction
Book 1 of 1: Fifty Shades Freed (Fifty Shades #3)
Link: books.toscrape.com/catalogue/fifty-shades-freed-fifty-shades-3_156/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 15.36
Availability: In stock
--------------------------------------------------------------------------------
No more books in this category.
---------------------------------------------------------------------------
Category: Humor
Book 1 of 10: The Long Haul (Diary of a Wimpy Kid #9)
Link: books.toscrape.com/

Category: History
Book 1 of 18: Sapiens: A Brief History of Humankind
Link: books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 54.23
Availability: In stock
--------------------------------------------------------------------------------
Category: History
Book 2 of 18: Unbound: How Eight Technologies Made Us Human, Transformed Society, and Brought Our World to the Brink
Link: books.toscrape.com/catalogue/unbound-how-eight-technologies-made-us-human-transformed-society-and-brought-our-world-to-the-brink_950/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 25.52
Availability: In stock
--------------------------------------------------------------------------------
Category: History
Book 3 of 18: The Age of Genius: The Seventeenth Century and the Birth of the Modern Mind
Link: books.toscrape.com/catalogue/the-age-of-genius-the-seventeenth-century-and-the-birth-of-the-modern-mind_929/index.html
R

Category: Food and Drink
Page: 2
Book 21 of 30: How to Cook Everything Vegetarian: Simple Meatless Recipes for Great Food (How to Cook Everything)
Link: books.toscrape.com/catalogue/how-to-cook-everything-vegetarian-simple-meatless-recipes-for-great-food-how-to-cook-everything_471/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 46.01
Availability: In stock
---------------------------------------------------------------------------
Category: Food and Drink
Page: 2
Book 22 of 30: How to Be a Domestic Goddess: Baking and the Art of Comfort Cooking
Link: books.toscrape.com/catalogue/how-to-be-a-domestic-goddess-baking-and-the-art-of-comfort-cooking_470/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 28.25
Availability: In stock
---------------------------------------------------------------------------
Category: Food and Drink
Page: 2
Book 23 of 30: The Barefoot Contessa Cookbook
Link: books.toscrape.com/catalogue/the-barefoot-contessa-cookbook_440/index.html


Category: Thriller
Book 1 of 11: In Her Wake
Link: books.toscrape.com/catalogue/in-her-wake_980/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 12.84
Availability: In stock
--------------------------------------------------------------------------------
Category: Thriller
Book 2 of 11: The Elephant Tree
Link: books.toscrape.com/catalogue/the-elephant-tree_968/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 23.82
Availability: In stock
--------------------------------------------------------------------------------
Category: Thriller
Book 3 of 11: Behind Closed Doors
Link: books.toscrape.com/catalogue/behind-closed-doors_962/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 52.22
Availability: In stock
--------------------------------------------------------------------------------
Category: Thriller
Book 4 of 11: You (You #1)
Link: books.toscrape.com/catalogue/you-you-1_784/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 43.61
A

Category: Christian
Book 1 of 3: (Un)Qualified: How God Uses Broken People to Do Big Things
Link: books.toscrape.com/catalogue/unqualified-how-god-uses-broken-people-to-do-big-things_873/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 54.00
Availability: In stock
--------------------------------------------------------------------------------
Category: Christian
Book 2 of 3: Crazy Love: Overwhelmed by a Relentless God
Link: books.toscrape.com/catalogue/crazy-love-overwhelmed-by-a-relentless-god_682/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 47.72
Availability: In stock
--------------------------------------------------------------------------------
Category: Christian
Book 3 of 3: Blue Like Jazz: Nonreligious Thoughts on Christian Spirituality
Link: books.toscrape.com/catalogue/blue-like-jazz-nonreligious-thoughts-on-christian-spirituality_461/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 25.77
Availability: In stock
-----------------

In [4]:
# Check the updated book_info
from pprint import pprint
pprint(book_info)

{'availability': ['In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
                  'In stock',
          

               '50.59',
               '18.88',
               '42.40',
               '24.96',
               '32.36',
               '38.43',
               '38.00',
               '37.59',
               '35.67',
               '43.30',
               '10.65',
               '26.12',
               '48.74',
               '21.36',
               '32.42',
               '10.92',
               '19.07',
               '33.26',
               '54.86',
               '51.48',
               '44.18',
               '36.26',
               '34.96',
               '44.84',
               '51.22',
               '24.42',
               '44.91',
               '40.44',
               '19.09',
               '23.89',
               '40.76',
               '14.07',
               '53.90',
               '29.06',
               '28.26',
               '25.91',
               '37.40',
               '40.67',
               '48.78',
               '37.61',
               '22.13',
               '

## Bringing it all together with Pandas

![Pandas](images/pandas.svg)

## What is Pandas?

- Popular Python library for data science and analytics
- Organizes Python data structures into tables called **DataFrames**.
- Super powerful, can export and import data from many different sources
- Can run any tabular function your heart desires

## Exporting our scraped data into a CSV file

In [5]:
import pandas as pd

# Construct a dataframe using our meal information
df = pd.DataFrame(book_info)

df

,title,url,num_stars,price_gbp,availability
0,It's Only the Himalayas,books.toscrape.com/catalogue/its-only-the-hima...,2,45.17,In stock
1,Full Moon over Noah’s Ark: An Odyssey to Mount...,books.toscrape.com/catalogue/full-moon-over-no...,4,49.43,In stock
2,See America: A Celebration of Our National Par...,books.toscrape.com/catalogue/see-america-a-cel...,3,48.87,In stock
3,Vagabonding: An Uncommon Guide to the Art of L...,books.toscrape.com/catalogue/vagabonding-an-un...,2,36.94,In stock
4,Under the Tuscan Sun,books.toscrape.com/catalogue/under-the-tuscan-...,3,37.33,In stock
...,...,...,...,...,...
995,Why the Right Went Wrong: Conservatism--From G...,books.toscrape.com/catalogue/why-the-right-wen...,4,52.65,In stock
996,Equal Is Unfair: America's Misguided Fight Aga...,books.toscrape.com/catalogue/equal-is-unfair-a...,1,56.86,In stock
997,Amid the Chaos,books.toscrape.com/catalogue/amid-the-chaos_78...,1,36.58,In stock
998,Dark Notes,books.toscrape.com/catalogue/dark-notes_800/in...,5,19.19,In stock


In [6]:
# Export to CSV file
df.to_csv("books.csv", index=False)